In [1]:
import numpy as np
import pandas as pd
from time import time
from sklearn import metrics
from collections import defaultdict
import matplotlib.pyplot as plt
import glob


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cluster import KMeans
from sklearn import preprocessing
from sklearn.metrics.cluster import contingency_matrix, homogeneity_score, completeness_score, adjusted_rand_score, adjusted_mutual_info_score, v_measure_score
from sklearn.metrics.cluster import contingency_matrix

## Import libraries
import re
import nltk
from nltk import pos_tag
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
#nltk.download('stopwords' )

import string
from string import punctuation
import matplotlib.pyplot as plt

In [2]:
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lex_rank import LexRankSummarizer
import random
import pandas as pd
import numpy as np
import re, json, datetime, pytz
import matplotlib.pyplot as plt
from sklearn.feature_extraction import text
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from textblob import TextBlob
from sklearn.ensemble import RandomForestClassifier
from collections import Counter
from collections import Counter
import nltk

/usr/lib/python3/dist-packages/requests/__init__.py:89: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (3.0.4) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


In [3]:
import pytz
import datetime

pst_tz = pytz.timezone('America/Los_Angeles')

In [4]:
from happytransformer import HappyTextClassification
happy_tc = HappyTextClassification(model_type="DISTILBERT", 
                                       model_name="distilbert-base-uncased-finetuned-sst-2-english", num_labels=2)

03/13/2022 05:20:30 - INFO - happytransformer.happy_transformer -   Using model: cuda


In [5]:
from transformers import pipeline
pipe = pipeline(task='text2text-generation', model='facebook/m2m100_418M')

In [6]:
def transformer_sentiment(tweet):
    
    result = happy_tc.classify_text(tweet)
    if result.label == 'NEGATIVE':
        return -1, -result.score
    else:
        return 1, result.score
    

In [7]:
def translate_and_transformer_sentiment(tweet):
    
    tweet = pipe(tweet, forced_bos_token_id=pipe.tokenizer.get_lang_id(lang='en'))
    tweet = tweet[0]['generated_text']
    result = happy_tc.classify_text(tweet)
    if result.label == 'NEGATIVE':
        return -1, -result.score
    else:
        return 1, result.score

In [8]:
def clean_tweet(tweet):
    '''
    Utility function to clean the text in a tweet by removing 
    links and special characters using regex.
    '''
    return ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)", " ", tweet).split())

In [9]:
def get_data(text):
    with open(text+'.txt', 'r',encoding="utf8") as f:
        retweet_count, fav_count, followers, place, title, time = [],[],[],[],[],[]
        sentiment_polarity, sentiment_score = [], []
        pst_tz = pytz.timezone('US/Pacific') 
        for line in f:
            tweet = json.loads(line)
            title.append(tweet['title'])
            retweet_count.append(tweet['metrics']['citations']['total']) 
            fav_count.append(tweet['tweet']['favorite_count'])
            followers.append(tweet['author']['followers'])
            time.append(datetime.datetime.fromtimestamp(tweet['citation_date'], pst_tz))
            place.append(tweet['tweet']['user']['location'])
            cleaned_tweet = clean_tweet(tweet['title'])
            if not tweet['tweet']['lang'] == 'en':
                senti_polarity, senti_score = translate_and_transformer_sentiment(cleaned_tweet)
                sentiment_polarity.append(senti_polarity)
                sentiment_score.append(senti_score)
            else:
                senti_polarity, senti_score = transformer_sentiment(cleaned_tweet)
                sentiment_polarity.append(senti_polarity)
                sentiment_score.append(senti_score)
            break
    d = {'title':title, 'retweet_count':retweet_count, 'favourite_count': fav_count, 'followers':followers,
         'time':time,'place':place, 'sentiment_polarity':sentiment_polarity, 'sentiment_score':sentiment_score}
    df = pd.DataFrame(d)
    return df   

In [10]:
def get_player(text, player=['tom', 'brady']):
    with open(text+'.txt', 'r',encoding="utf8") as f:
        retweet_count, fav_count, followers, place, title, time = [],[],[],[],[],[]
        sentiment_polarity, sentiment_score = [], []
        pst_tz = pytz.timezone('US/Pacific') 
        for line in f:
            tweet = json.loads(line)
            tweet_text = tweet['title']
            tweet_text = tweet_text.lower().split()
            for word in player: 
                if word in tweet_text:
                    title.append(tweet['title'])
                    retweet_count.append(tweet['metrics']['citations']['total']) 
                    fav_count.append(tweet['tweet']['favorite_count'])
                    followers.append(tweet['author']['followers'])
                    time.append(datetime.datetime.fromtimestamp(tweet['citation_date'], pst_tz))
                    place.append(tweet['tweet']['user']['location'])
                    cleaned_tweet = clean_tweet(tweet['title'])
                    if not tweet['tweet']['lang'] == 'en':
                        senti_polarity, senti_score = translate_and_transformer_sentiment(cleaned_tweet)
                        sentiment_polarity.append(senti_polarity)
                        sentiment_score.append(senti_score)
                    else:
                        senti_polarity, senti_score = transformer_sentiment(cleaned_tweet)
                        sentiment_polarity.append(senti_polarity)
                        sentiment_score.append(senti_score)
                    break
    d = {'title':title, 'retweet_count':retweet_count, 'favourite_count': fav_count, 'followers':followers,
         'time':time,'place':place, 'sentiment_polarity':sentiment_polarity, 'sentiment_score':sentiment_score}
    df = pd.DataFrame(d)
    return df   

In [11]:
df = get_player('tweets_#gopatriots')

/home/shivam/.local/lib/python3.8/site-packages/transformers/pipelines/base.py:997: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


In [60]:
Limdf= df[(df['time'] > '2015-02-01 15:00:00') & (df['time'] < '2015-02-02 10:00:00')]
times = pd.to_datetime(Limdf.time)
grouped = df.groupby([times.dt.date, times.dt.hour]).title.transform(lambda x: ',,,'.join(x))


In [61]:
pd.options.display.max_colwidth = 1000
print(grouped.head(1))

180    Tom Brady is an American Legend. #BeatTheSeahawks #GoPatriots http://t.co/0afqnbLH1I,,,Statement from a guy at my house: Gronk is the guy you wanna party with. Brady is the guy you bring home to mom.  #GOPATRIOTS #SuperBowI,,,En mi TL nada más que hay mensajes de ánimo a los seahawks pero yo soy muy Brady asiq #GoPatriots,,,Brady looks pumped!!! About two pounds less than full though🏈👌#gopatriots http://t.co/fehtxEYTzQ,,,Tom Brady has a lot to prove #GoPatriots,,,In Tom Brady we trust #GoPatriots #PatsNation #NePatriots #NEvsSEA #SuperBowlXLIX #HappyBday2Me,,,Go Patriots, Brady mostra para eles quem é que manda #SuperBowlNoEsporteInterativo  #GoPatriots #GoBrady,,,Secundária do Patriots muito bem só espero que o Brady não sinta a interceptação #GoPatriots #SuperBowlNoEsporteInterativo,,,Tom Brady is my dream man!  #gopatriots,,,Go Brady 😍🙈🏈💙. \n#GoPatriots http://t.co/9J8Nh87CTA,,,#C4NFL I'm new to American Football, but I'm really enjoying it :) Especially the dishy QB Brady ;)

In [30]:
import transformers

In [31]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)


Downloading:   0%|          | 0.00/1.76k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

In [62]:
for index, value in grouped.items():
    to_tokenize = value
    print(value)
    break

#     summarized = summarizer(to_tokenize, min_length=5, max_length=50)
#     summ=' '.join([str(i) for i in summarized])
#     summ=summ.replace("{","")
#     summ=summ.replace("''","")
#     summ=summ.replace("\x92","")
#     print(summ)

Tom Brady is an American Legend. #BeatTheSeahawks #GoPatriots http://t.co/0afqnbLH1I,,,Statement from a guy at my house: Gronk is the guy you wanna party with. Brady is the guy you bring home to mom.  #GOPATRIOTS #SuperBowI,,,En mi TL nada más que hay mensajes de ánimo a los seahawks pero yo soy muy Brady asiq #GoPatriots,,,Brady looks pumped!!! About two pounds less than full though🏈👌#gopatriots http://t.co/fehtxEYTzQ,,,Tom Brady has a lot to prove #GoPatriots,,,In Tom Brady we trust #GoPatriots #PatsNation #NePatriots #NEvsSEA #SuperBowlXLIX #HappyBday2Me,,,Go Patriots, Brady mostra para eles quem é que manda #SuperBowlNoEsporteInterativo  #GoPatriots #GoBrady,,,Secundária do Patriots muito bem só espero que o Brady não sinta a interceptação #GoPatriots #SuperBowlNoEsporteInterativo,,,Tom Brady is my dream man!  #gopatriots,,,Go Brady 😍🙈🏈💙. 
#GoPatriots http://t.co/9J8Nh87CTA,,,#C4NFL I'm new to American Football, but I'm really enjoying it :) Especially the dishy QB Brady ;) #GoPatr

In [37]:
type(value)

str

In [63]:
to_tokenize = value.split(',,,')

In [64]:
to_tokenize

['Tom Brady is an American Legend. #BeatTheSeahawks #GoPatriots http://t.co/0afqnbLH1I',
 'Statement from a guy at my house: Gronk is the guy you wanna party with. Brady is the guy you bring home to mom.  #GOPATRIOTS #SuperBowI',
 'En mi TL nada más que hay mensajes de ánimo a los seahawks pero yo soy muy Brady asiq #GoPatriots',
 'Brady looks pumped!!! About two pounds less than full though🏈👌#gopatriots http://t.co/fehtxEYTzQ',
 'Tom Brady has a lot to prove #GoPatriots',
 'In Tom Brady we trust #GoPatriots #PatsNation #NePatriots #NEvsSEA #SuperBowlXLIX #HappyBday2Me',
 'Go Patriots, Brady mostra para eles quem é que manda #SuperBowlNoEsporteInterativo  #GoPatriots #GoBrady',
 'Secundária do Patriots muito bem só espero que o Brady não sinta a interceptação #GoPatriots #SuperBowlNoEsporteInterativo',
 'Tom Brady is my dream man!  #gopatriots',
 'Go Brady 😍🙈🏈💙. \n#GoPatriots http://t.co/9J8Nh87CTA',
 "#C4NFL I'm new to American Football, but I'm really enjoying it :) Especially the di

In [ ]:
summarized = summarizer(to_tokenize , min_length=5, max_length=50)

Your max_length is set to 50, but you input_length is only 33. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 50, but you input_length is only 40. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)
Your max_length is set to 50, but you input_length is only 38. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=19)
Your max_length is set to 50, but you input_length is only 36. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=18)
Your max_length is set to 50, but you input_length is only 13. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=6)
Your max_length is set to 50, but you input_length is only 35. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your max_length is set to 50, but you input_length is only 37. You might consider decreas

Your max_length is set to 50, but you input_length is only 32. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 50, but you input_length is only 46. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=23)
Your max_length is set to 50, but you input_length is only 20. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=10)
Your max_length is set to 50, but you input_length is only 15. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=7)
Your max_length is set to 50, but you input_length is only 22. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=11)
Your max_length is set to 50, but you input_length is only 32. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 50, but you input_length is only 14. You might consider decreas

Your max_length is set to 50, but you input_length is only 32. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 50, but you input_length is only 35. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=17)
Your max_length is set to 50, but you input_length is only 10. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=5)
Your max_length is set to 50, but you input_length is only 27. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=13)
Your max_length is set to 50, but you input_length is only 30. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=15)
Your max_length is set to 50, but you input_length is only 32. You might consider decreasing max_length manually, e.g. summarizer('...', max_length=16)
Your max_length is set to 50, but you input_length is only 34. You might consider decreas